# Split data into low vs high RRM2B cohorts for GSEA analysis

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np # scientific computing
import pandas as pd # data loading and processing
import os # os operations
import matplotlib.pyplot as plt # for generating figures
import math
import matplotlib.dates as mdates
import seaborn as sns # for generating visualizations, better support with pandas than matplotlib
from scipy import stats
import csv
from sklearn.impute import SimpleImputer
from analysis import *

In [2]:
# load pancan data manually as we don't want to load HCCDB data

tcga = pd.read_csv("./data/EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp (1).xena", index_col = 0, sep = "\t") # gene x patient
pheno = pd.read_csv("./data/TCGA_phenotype_denseDataOnlyDownload (1).tsv", index_col = 0, sep = "\t") # patient x phenotype

# attach cancer type to each patient
data = tcga.T
data = pd.concat([data, pheno], axis = 1, join = "inner") # patients x genes

# attach abbeviations for each cancer type
ls = data["_primary_disease"].unique().tolist()

conditions = [
    data['_primary_disease'] == 'adrenocortical cancer',
    data['_primary_disease'] == 'bladder urothelial carcinoma',
    data['_primary_disease'] == 'breast invasive carcinoma',
    data['_primary_disease'] == 'cervical & endocervical cancer',
    data['_primary_disease'] == 'cholangiocarcinoma', 
    data['_primary_disease'] == 'colon adenocarcinoma',
    data['_primary_disease'] == 'diffuse large B-cell lymphoma',
    data['_primary_disease'] == 'esophageal carcinoma',
    data['_primary_disease'] == 'glioblastoma multiforme',
    data['_primary_disease'] == 'head & neck squamous cell carcinoma',
    data['_primary_disease'] == 'kidney chromophobe',
    data['_primary_disease'] == 'kidney clear cell carcinoma',
    data['_primary_disease'] == 'kidney papillary cell carcinoma',
    data['_primary_disease'] == 'acute myeloid leukemia',
    data['_primary_disease'] == 'brain lower grade glioma',
    data['_primary_disease'] == 'liver hepatocellular carcinoma',
    data['_primary_disease'] == 'lung adenocarcinoma',
    data['_primary_disease'] == 'lung squamous cell carcinoma',
    data['_primary_disease'] == 'mesothelioma',
    data['_primary_disease'] == 'ovarian serous cystadenocarcinoma',
    data['_primary_disease'] == 'pancreatic adenocarcinoma',
    data['_primary_disease'] == 'pheochromocytoma & paraganglioma',
    data['_primary_disease'] == 'prostate adenocarcinoma',
    data['_primary_disease'] == 'rectum adenocarcinoma',
    data['_primary_disease'] == 'sarcoma',
    data['_primary_disease'] == 'skin cutaneous melanoma',
    data['_primary_disease'] == 'stomach adenocarcinoma',
    data['_primary_disease'] == 'testicular germ cell tumor',
    data['_primary_disease'] == 'thyroid carcinoma',
    data['_primary_disease'] == 'thymoma',
    data['_primary_disease'] == 'uterine corpus endometrioid carcinoma',
    data['_primary_disease'] == 'uterine carcinosarcoma',
    data['_primary_disease'] == 'uveal melanoma'    
]

choices = ["ACC",
           "BLCA",
           "BRCA",
           "CESC",
           "CHOL",
           "COAD",
           "DBLC",
           "ESCA",
           "GBM",
           "HNSC",
           "KICH",
           "KIRC",
           "KIRP",
           "LAML",
           "LGG",
           "LIHC",
           "LUAD",
           "LUSC",
           "MESO",
           "OV",
           "PAAD",
           "PCPG",
           "PRAD",
           "READ",
           "SARC",
           "SKCM",
           "STAD",
           "TGCT",
           "THCA",
           "THYM",
           "UCEC",
           "UCS",
           "UVM"
           ]

data["ptype"] = np.select(conditions, choices, default = "null")
data.head()

,100130426,100133144,100134869,10357,10431,136542,155060,26823,280660,317712,...,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,sample_type_id,sample_type,_primary_disease,ptype
TCGA-OR-A5J1-01,0.0,2.09,2.30,7.23,10.99,0.0,8.10,1.29,0.0,0.0,...,10.04,0.57,9.34,10.85,10.18,9.22,1.0,Primary Tumor,adrenocortical cancer,ACC
TCGA-OR-A5J2-01,0.0,1.88,3.32,6.36,10.35,0.0,7.65,0.00,0.0,0.0,...,11.54,5.02,10.19,11.58,10.89,9.65,1.0,Primary Tumor,adrenocortical cancer,ACC
TCGA-OR-A5J3-01,0.0,1.45,2.92,6.45,10.04,0.0,8.45,0.67,0.0,0.0,...,9.84,0.67,9.66,11.38,10.53,8.78,1.0,Primary Tumor,adrenocortical cancer,ACC
TCGA-OR-A5J5-01,0.0,0.00,1.35,5.78,11.20,0.0,8.78,0.83,0.0,0.0,...,9.80,3.66,9.12,11.21,10.16,9.01,1.0,Primary Tumor,adrenocortical cancer,ACC
TCGA-OR-A5J6-01,0.0,0.00,2.45,6.09,10.30,0.0,7.23,0.00,0.0,0.0,...,9.81,3.14,9.64,9.47,9.64,8.90,1.0,Primary Tumor,adrenocortical cancer,ACC


In [ ]:
data.shape

In [22]:
#script to find genes with log fold change >= 0.32

databases = ['STAD'] #get_db_for_single_gene_analysis("./gene_set_for_single_gene_analysis.txt")
# databases = ['PANCAN']  #, 'DBLC', 'SKCM', 'HNSC', 'PRAD', 'PAAD', 'SARC', 
             #'BRCA', 'UCS', 'ESCA', 'STAD', 'LAML', 'OV', 'PANCAN'

df = pd.DataFrame()

for db in databases:
    #init
    print(db)
    h = []
    l = []

    # load data
    print("loading data")
    df = extract_rows_by_type(data, hccdb=None, db=db)
    
    # impute missing values as GSEA cannot handle them
    df = impute_nan(df)

    # bin the patients into quartiles based on RRM2B expression
    print("binning patients")
    iqr = df["RRM2B"].describe()
    df["RRM2B_levels"] = pd.cut(df["RRM2B"],
                    bins=[ iqr["min"], iqr["25%"], iqr["75%"], iqr["max"]],
                    labels=["Bottom 25%", "-", "Top 25%"])
    df.drop(df.loc[df["RRM2B_levels"]=="-"].index, inplace=True)

    # extract the top and bottom 25% of patients
    print("grouping patients")
    high = df[df["RRM2B_levels"] == "Top 25%"]
    low = df[df["RRM2B_levels"] == "Bottom 25%"]
    print("high", high.shape)
    print("low:", low.shape)
    high.drop("RRM2B_levels", axis = 1, inplace=True)
    low.drop("RRM2B_levels", axis = 1, inplace=True)
    high.drop_duplicates(inplace = True)
    low.drop_duplicates(inplace = True)
    out = pd.concat([high, low]).T

    # export data
    print("exporting data")
    out.to_csv(str(db) + " _expression_1.25_GSEA.csv")

    # export phenotypes
    h = ["high" for i in range(high.shape[0])]
    l = ["low" for i in range(low.shape[0])]
    gsea_phenotypes = pd.concat([pd.DataFrame(h).T, pd.DataFrame(l).T], axis = 1)
    gsea_phenotypes.to_csv(str(db) + " _pheno_1.25_GSEA.csv")



LIHC
loading data
LIHC
imputing data
transpose
impute
done imputing
binning patients
grouping patients
high (104, 20532)
low: (106, 20532)


/var/folders/mp/lfr86vf57gzgqs4tr6qd5bs80000gn/T/ipykernel_2624/37289725.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high.drop("RRM2B_levels", axis = 1, inplace=True)
/var/folders/mp/lfr86vf57gzgqs4tr6qd5bs80000gn/T/ipykernel_2624/37289725.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low.drop("RRM2B_levels", axis = 1, inplace=True)
/var/folders/mp/lfr86vf57gzgqs4tr6qd5bs80000gn/T/ipykernel_2624/37289725.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

exporting data


In [ ]:
# # script to find genes with log fold change >= 0.58

# databases = ['STAD' ]  #, 'DBLC', 'SKCM', 'HNSC', 'PRAD', 'PAAD', 'SARC', 
#              #'BRCA', 'UCS', 'ESCA', 'STAD', 'LAML', 'OV', 'PANCAN'

# df = pd.DataFrame()

# for db in databases:
#     print(db)
    
#     # load data
#     print("loading data")
#     if db == "PANCAN":
#         df = data
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)
#     else:
#         df = data[data["ptype"] == db]
#         df = df.T # genes x patients
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)
    
#     df = impute_nan(df)

#     # bin the patients into quartiles based on RRM2B expression
#     print("binning patients")
#     iqr = df["RRM2B"].describe()
#     df["RRM2B_levels"] = pd.cut(df["RRM2B"],
#                     bins=[ iqr["min"], iqr["25%"], iqr["75%"], iqr["max"]],
#                     labels=["Bottom 25%", "-", "Top 25%"])
#     df.drop(df.loc[df["RRM2B_levels"]=="-"].index, inplace=True)

#     # group patients into high and low RRM2B expression
#     print("grouping patients")
#     high = df[df["RRM2B_levels"] == "Top 25%"]
#     low = df[df["RRM2B_levels"] == "Bottom 25%"]
#     print("high", high.shape)
#     print("low:", low.shape)
#     high.drop("RRM2B_levels", axis = 1, inplace=True)
#     low.drop("RRM2B_levels", axis = 1, inplace=True)
#     out = pd.concat([high, low]).T

#     print("exporting data")
#     out.to_csv(db + " _expression_1.25_GSEA.csv")

